In [1]:
import pandas as pd
import numpy as np

import mysql.connector
from sqlalchemy import create_engine
import pandas.io.sql as psql
import sqlalchemy

from dateutil import parser
from datetime import datetime
from datetime import date
from dateutil.relativedelta import relativedelta


In [2]:
database_username = 'root'
database_password = 'yash'
database_ip       = '127.0.0.1'
database_name     = 'mainschema'
database_connection = sqlalchemy.create_engine('mysql+mysqlconnector://{0}:{1}@{2}/{3}'.
                                               format(database_username, database_password, 
                                                      database_ip, database_name))

In [3]:
df = pd.read_sql('SELECT * FROM marketsmojorecos', con=database_connection)

In [4]:
df

,srno,index,broker,company_name,current_price,recomended_buying,predict_date,target_price
0,1,0,Prabhudas Lilladher,Colgate Palmol. (I),1248.25,1274.35,2019-09-03,1159.00
1,2,1,Axis Direct,Larsen & Toubro,1310.90,1328.1,2019-09-03,1487.00
2,3,2,Nirmal Bang,Britannia Inds,2695.00,2704.8,2019-09-03,3220.00
3,4,3,Sushil Finance,Sahyadri Industries,147.00,145.5,2019-09-03,220.00
4,5,4,Geojit Financial Services,Tata Chemicals,580.95,579.06,2019-08-30,531.00
...,...,...,...,...,...,...,...,...
22237,22238,3526,Prabhudas Lilladher,BPCL,493.00,378.64,2019-09-05,466.00
22238,22239,3527,Axis Direct,Bajaj Finance,4056.35,3243.58,2019-09-04,3591.00
22239,22240,3528,Geojit Financial Services,Hindalco,207.60,179.21,2019-09-04,195.00
22240,22241,3529,Geojit Financial Services,HDFC,2368.00,2096.68,2019-09-04,2284.00


In [5]:
print(len(df))


22242


In [6]:
start_date = '2016-01-01'
end_date = '2016-12-31'

In [7]:
date_object1 = datetime.strptime(start_date, '%Y-%m-%d').date()
print(type(date_object1))
print(date_object1)

<class 'datetime.date'>
2016-01-01


In [8]:
date_object2 = datetime.strptime(end_date, '%Y-%m-%d').date()
print(type(date_object2))
print(date_object2)

<class 'datetime.date'>
2016-12-31


In [9]:
mask = (df['predict_date'] > date_object1) & (df['predict_date'] <= date_object2)

# 2016 data

In [10]:
df1 = df.loc[mask]
df1 = df1.reset_index()
del df1['level_0']
del df1['srno']
del df1['index']
df1

,broker,company_name,current_price,recomended_buying,predict_date,target_price
0,KRChoksey,Force Motors,1199.90,3497.00,2016-10-03,4820.00
1,AUM Capital,Narayana Hrudayalay,239.00,325.55,2016-09-27,400.00
2,Religare,ICICI Bank,390.15,217.73,2016-08-04,181.82
3,Enochventures.com,Parag Milk Foods,137.65,329.6,2016-07-12,425.00
4,Edelweiss,Kewal Kiran Clothing,1000.00,1796.00,2016-05-25,2039.00
...,...,...,...,...,...,...
5193,Axis Direct,Cox & Kings India,3.61,179.00,2016-04-06,338.00
5194,Dolat Capital,Mphasis,966.20,454.00,2016-04-05,620.00
5195,Angel Broking,Dewan Hsg Fin. Corp.,47.15,189.00,2016-04-05,270.00
5196,Angel Broking,MBL Infrastructures,5.57,154.00,2016-04-04,285.00


In [11]:
print("No of Brokers in 2016 :",df1['broker'].nunique()) 

No of Brokers in 2016 : 61


In [12]:
s1=len(df1)
s1

5198

In [13]:
df3=pd.DataFrame()
for i in range(0,5198): 
    sym=df1.at[i, 'company_name']
    sym = sym.lower()
    sym = sym.replace(" ","")
    sym = sym.replace(".","")
    sym = sym.replace("(i)","")
    sym = sym.replace("(","")
    sym = sym.replace(")","")
    sym = sym.replace("-","")
    sqlrun = "SELECT SYMBOL FROM symbolsmapped WHERE recoName = '"+sym+"'"
    df3 = df3.append(pd.read_sql(sqlrun, con=database_connection),ignore_index=True)

In [14]:
df3

,SYMBOL
0,FORCEMOT
1,NH
2,ICICIBANK
3,PARAGMILK
4,KKCL
...,...
5193,CKFSL
5194,MPHASIS
5195,None
5196,None


In [15]:
df4 =pd.concat([df1,df3],axis=1)
df4

,broker,company_name,current_price,recomended_buying,predict_date,target_price,SYMBOL
0,KRChoksey,Force Motors,1199.90,3497.00,2016-10-03,4820.00,FORCEMOT
1,AUM Capital,Narayana Hrudayalay,239.00,325.55,2016-09-27,400.00,NH
2,Religare,ICICI Bank,390.15,217.73,2016-08-04,181.82,ICICIBANK
3,Enochventures.com,Parag Milk Foods,137.65,329.6,2016-07-12,425.00,PARAGMILK
4,Edelweiss,Kewal Kiran Clothing,1000.00,1796.00,2016-05-25,2039.00,KKCL
...,...,...,...,...,...,...,...
5193,Axis Direct,Cox & Kings India,3.61,179.00,2016-04-06,338.00,CKFSL
5194,Dolat Capital,Mphasis,966.20,454.00,2016-04-05,620.00,MPHASIS
5195,Angel Broking,Dewan Hsg Fin. Corp.,47.15,189.00,2016-04-05,270.00,None
5196,Angel Broking,MBL Infrastructures,5.57,154.00,2016-04-04,285.00,None


In [16]:
sym=df4.at[1, 'SYMBOL']
sym

'NH'

In [18]:
mydate1=df4.at[1, 'predict_date']#prediction date
print(mydate1)

mydate2 = mydate1 + relativedelta(years=1)#date 1 year after prediction
print(mydate2)

dbyear1 = "nse"+str(mydate1.year)
print(dbyear1)

dbyear2 = "nse"+str(mydate2.year)
print(dbyear2)

2016-09-27
2016-12-31
2017-09-27
2017-01-01
nse2016
nse2017


In [36]:
sqlrun1 = "SELECT * FROM "+dbyear1+" WHERE Date >= '"+str(mydate1)+"' AND Symbol = '"+sym+"'"
sqlrun1

"SELECT * FROM nse2016 WHERE Date >= '2016-09-27' AND Symbol = 'NH'"

In [37]:
dfprices1 = pd.read_sql(sqlrun1, con=database_connection)
del dfprices1['index']
dfprices1

,Symbol,Date,Closeval
0,NH,2016-09-27,333.45
1,NH,2016-09-28,336.05
2,NH,2016-09-29,323.50
3,NH,2016-09-30,333.55
4,NH,2016-10-03,343.60
...,...,...,...
60,NH,2016-12-26,344.20
61,NH,2016-12-27,338.85
62,NH,2016-12-28,333.80
63,NH,2016-12-29,335.05


In [38]:
sqlrun2 = "SELECT * FROM "+dbyear2+" WHERE Date <= '"+str(mydate2)+"' AND Symbol = '"+sym+"'"
sqlrun2

"SELECT * FROM nse2017 WHERE Date <= '2017-09-27' AND Symbol = 'NH'"

In [39]:
dfprices2 = pd.read_sql(sqlrun2, con=database_connection)
del dfprices2['index']
dfprices2

,Symbol,Date,Closeval
0,NH,2017-01-02,344.35
1,NH,2017-01-03,339.45
2,NH,2017-01-04,334.90
3,NH,2017-01-05,339.65
4,NH,2017-01-06,339.65
...,...,...,...
178,NH,2017-09-21,300.80
179,NH,2017-09-22,299.15
180,NH,2017-09-25,300.35
181,NH,2017-09-26,300.20


In [40]:
dfprices = pd.concat([dfprices1,dfprices2],ignore_index=True)

In [41]:
dfprices

,Symbol,Date,Closeval
0,NH,2016-09-27,333.45
1,NH,2016-09-28,336.05
2,NH,2016-09-29,323.50
3,NH,2016-09-30,333.55
4,NH,2016-10-03,343.60
...,...,...,...
243,NH,2017-09-21,300.80
244,NH,2017-09-22,299.15
245,NH,2017-09-25,300.35
246,NH,2017-09-26,300.20
